# [10] CNN 돌리기 (CNN Activatioon)

In [1]:
# CIFAR10 데이터 
# 설명 : 10개 클래스  ,5만개 train image , 1만개 test image

# 맥락

1. `Imports` 라이브러리
2. `transforms data` 수행가능한 데이터 만들기(performing)
3. `batch size` 세팅하기 train,test 셋 다운로드하기.
4. validation 20% 로 대략설정하여 데이터를 sampler 로 나누기
5. `DataLoader()` 함수로 각 세트의 배치정하기
6. 구조 정의하기. (Define the architecture of your network)
7. 파라미터정의하기.(epoch 도 정의)
8. 실제로 모델 학습시키기. loss 와 accuracy 계산.
9. Plot 으로 그래프그리기.


## Data Augmentation (데이터증강)

`-` 중요성 : 1. 숫자가 많으면 학습이 잘된다.(데이터부족해결, 일반화 성능향상, 과적합 방지) 그래서 여러 변형을 주며 데이터증강을 할 필요가 있다.

# 코드

In [3]:
import numpy as np
import torch
from torch import nn,optim
import torch.nn.functional as F
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [9]:
transform = \
transforms.Compose([transforms.ToTensor,
                    transforms.Normalize((0.5,0.5,0.5),
                                         (0.5,0.5,0.5))])

In [12]:
batch_size=100

train_data =datasets.CIFAR10('data',train=True, 
                             download=True,
                             transform=transform)
test_data =datasets.CIFAR10('data',train=False, 
                             download=True,
                             transform=transform)


100.0%


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [14]:
dev_size = 0.2
idx = list(range(len(train_data)))
np.random.shuffle(idx)
split_size = int(np.floor(dev_size * len(train_data)))
train_idx,dev_idx = idx[split_size:],idx[:split_size]

train_sampler = SubsetRandomSampler(train_idx) #훈련셋 무작위 선택
dev_sampler = SubsetRandomSampler(dev_idx)

# 현재과정은 test 셋은 나두고 train셋을 8:2 로 나누어
#dev 셋에서 학습 성능을 검증하기 위해 만든 세트입니다.

In [15]:
train_loader = \
torch.utils.data.DataLoader(train_data,
                            batch_size=batch_size,
                            sampler=train_sampler)
dev_loader = \
torch.utils.data.DataLoader(train_data,
                            batch_size=batch_size,
                            sampler=dev_sampler)
test_loader = \
torch.utils.data.DataLoader(test_data,
                            batch_size=batch_size)

#이게 기초
```python
class CNN(nn.Module): #Pytorch 의 모든 신경망 모델의 상속 모델이다.
    def __init__(self):
        super(CNN, self).__init__()
        # 모델의 레이어들을 여기에 정의

    def forward(self, x):
        # 순전파 동작을 여기에 정의
        return x
```

In [17]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # 모델의 레이어들을 여기에 정의
        self.conv1 = nn.Conv2d(3,10,3,1,1)
        self.conv2 = nn.Conv2d(10,20,3,1,1)
        self.conv3 = nn.Conv2d(20,40,3,1,1)
        self.pool = nn.MaxPool2d(2,2)

        self.linear1 = nn.Linear(40*4*4,100)
        self.linear2 = nn.Linear(100,10)
        self.dropout = nn.Dropout(0.2)

    
    def forward(self, x):
        # 순전파 동작을 여기에 정의
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))


        x = x.view(-1,40*4*4)
        x = self.dropout(x)
        x = F.relu(self.linear1(x))
        x = self.dropout(x)
        x = F.log_softmax(self.linear2(x), dim=1)
        return x


In [18]:
model = CNN()
loss_function = nn.NLLLose() #negative log-likelihood)
optimizer = optim.Adam(model.parameters(),lr=0.001)
epochs = 50

AttributeError: module 'torch.nn' has no attribute 'NLLLose'